In [ ]:
#Cargar las librerias necesarias para la ejecución del proyecto modelo de predicción de retiros voluntarios

import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
#Cargar la base de datos con la cual vamos a trabajar en el proyecto

retiros=pd.read_csv("C:\Users\jforero\OneDrive - Grupo Bancolombia\91. Universidad - Especialización\Proyecto_Final_Retiros_Voluntarios\Retiros_historicos.csv")


In [ ]:
retiros.info()

In [ ]:
pd.set_option("display.max_columns",None)              #con este código podemos observar todas las columnas del df.
retiros.sample(2)

In [ ]:
retiros["CLASE_DE_MEDIDA"].value_counts()     #Valores unicos por clase de medida



In [ ]:
retiros["MOTIVO_RETIRO"].value_counts()            #Valores unicos por motivo de retiro

In [ ]:
output = []
for col in retiros.columns:
    vc = retiros[col].value_counts(dropna=False).sort_index()
    total = vc.sum()
    output.append(f"{col}:\n{vc}\nTotal: {total}")
print('\n\n'.join(output))                                              #con esta formula se verifica todos los valores unicos a la vez, permitiendo visualizar si tenemos valores nulos, caracteres especiales en todas las columnas a la vez

# **VALORES NULOS, CARACTERES ESPECIALES Y EN BLANCO**

Se revisa y se identifica que las siguientes columnas no presentan carácteres especiales, ni en blancos, más sin embargo, se identifican algunos valores nulos. Para este ejercicio determinamos que las columnas con un porcentaje de valores nulos mayor al 30% será eliminado.  veamos:
1. Area= 11
2. Còdigo de area= 2
3. Còdigo de cargo= 22579***20%
4. VP actual= 12
5. Vp Ant= 18543***17%
6. 2 act= 12
7. 2 ant= 18543***17%
8. 3 act= 12
9. 3 ant= 18543***17%
10. 4 act= 12
11. 4 ant= 18543***17%
12. 5 act= 12
13. 5 ant= 18543***17%
14. Nivel de cargo= 3387***3%
15. red de sucursales= 3387***3%
16. Cargo anterior= 7201***7%
17. Compañia anterior= 27928***26%
18. Nivel de cargo anterior= 18769***17%


Se analizan cada una de las columnas y se determina que no se presentan valores en blanco, ni carácteres especiales. Más sin embargo, se identifican algunos valores nulos que representan menos del 30%, por lo que por este motivo no será eliminadas.


# **LIMPIEZA**

# VARIABLE GENERO
Se identifica que la columna: "Género presenta inconsistencia en sus datos, se escribe de diferentes formas, por lo que se procede a limpiar la columna a continuación.

In [ ]:
retiros['GÉNERO'] = retiros['GÉNERO'].str.strip().str.lower()   #.str.strip() Este método elimina los espacios en blanco al inicio y al final de cada valor de texto en la columna.

                                                                #.str.lower() Después de quitar espacios, este método convierte todo el texto a minúsculas.
retiros['GÉNERO'] = retiros['GÉNERO'].replace({
    'femenino': 'Femenino',
    'feménino': 'Femenino',
    'masculino': 'Masculino',
    'masculíno': 'Masculino',
})


print(retiros['GÉNERO'].unique())

# VARIABLE NIVEL DE CARGO
Se identifica que la columna: "Nivel de cargo" presenta inconsistencia en sus datos, se escribe de diferentes formas, por lo que se procede a limpiar la columna a continuación.

In [ ]:
retiros['NIVEL_DEL_CARGO'] = retiros['NIVEL_DEL_CARGO'].replace('PROFESIONALES', 'PROFESIONAL')


In [ ]:
retiros.info()

Se revisa el tipo de datos de las columnas a trabajar en el modelo y se identifica que todas presentan el formato correcto; más sin embargo, se identifica que la variable año, es pertienente convertirla a categorica, ya que con este dato no se pueden sacar estadisticas. Y, la columna: periodo se debe convertir a: DATATIME, ya que es una fecha, además se identifica la necesidad de crear una nueva columna, para la variable a predecir, en este caso: "Retiros voluntarios"

# **CREAR COLUMNA PARA RETIROS VOLUNTARIOS Y RETIROS VOLUNTARIOS POR ARREGLO MUTUO"**

Vamos  a clasificar la variable a predecir como "RETIRO VOLUNTARIO" en donde sus valores van a ser:

1 --> Verdadero: El empleado se retiro voluntariamente (en este valor van a estar relacionados de la variable MOTIVO  DE RETIRO, los valores de "RETIRO VOLUNTARIO" y "VOLUNTARIO ARREGLO MUTUO"

0 --> Falso: El empelado se retiro de su cargo por cualquier otro motivo  los cuales estan relacionados en la variable MOTIVO  DE RETIRO ≠ "RETIRO VOLUNTARIO" y "VOLUNTARIO ARREGLO MUTUO"

In [ ]:
retiros['RETIRO_VOLUNTARIO'] = np.where(
    retiros['MOTIVO_RETIRO'].isin(['RETIRO VOLUNTARIO', 'VOLUNTARIO ARREGLO MUTUO']),
    1,
    0
)

In [ ]:
print(retiros['RETIRO_VOLUNTARIO'].value_counts().sort_index())
print("Suma total frecuencia:", retiros['RETIRO_VOLUNTARIO'].value_counts(dropna=False).sum())

In [ ]:
retiros.sample(2)

In [ ]:
retiros.info()

# ESTADISTICA DESCRIPTIVA

In [ ]:
variables_modelo = [
    "COMPAÑÍA", "CODIGO_AREA", "AREA", "CODIGO_CARGO", "CARGO", "REGION",
    "FORMA_DE_TRABAJO", "AÑO", "PERIODO", "GÉNERO", "RETIRO_VOLUNTARIO", "NIVEL_DEL_CARGO","RED_DE_SUCURSALES"
]

In [ ]:
retiros_reducido = retiros[variables_modelo].copy()

Convertimos 'RETIRO_VOLUNTARIO' como BOOLEANA para poderlo representar mejor en un diagrama de barras

In [ ]:
retiros['RETIRO_VOLUNTARIO'] = retiros['RETIRO_VOLUNTARIO'].astype("bool")

In [ ]:
import plotly.express as px

# Reemplazar True/False por "Sí"/"No"
conteo = retiros_reducido['RETIRO_VOLUNTARIO'].replace({True: "Sí", False: "No"}).value_counts().reset_index()
conteo.columns = ['RETIRO_VOLUNTARIO', 'count']

# Crear gráfico de barras con etiquetas visibles
fig = px.bar(
    conteo,
    x='RETIRO_VOLUNTARIO',
    y='count',
    text='count',
    labels={'RETIRO_VOLUNTARIO': '¿Se retiró voluntariamente?', 'count': 'Cantidad'},
    title='Retiros voluntarios en Bancolombia',
    width=600,
    height=500  # Aumenta altura para dar más espacio
)

# Posición y estilo del texto
fig.update_traces(
    textposition='outside',
    textfont_size=14  # Tamaño más grande para mejor lectura
)

# Ajustar márgenes y diseño
fig.update_layout(
    margin=dict(t=100),  # Espacio superior extra
    yaxis_title='Cantidad',
    xaxis_title='¿Se retiró voluntariamente?',
    uniformtext_minsize=8,
    uniformtext_mode='show'
)

fig.show()

In [ ]:
import plotly.express as px

# Paso 1: Top 20 códigos de área
top_areas = retiros['CODIGO_AREA'].value_counts().head(20).index

# Paso 2: Filtrar los datos
df_top = retiros[retiros['CODIGO_AREA'].isin(top_areas)]

# Paso 3: Agrupar por código de área y retiro voluntario
grouped = df_top.groupby(['CODIGO_AREA', 'RETIRO_VOLUNTARIO'], observed=True).size().reset_index(name='Cantidad')

# Paso 4: Convertir booleanos a texto
grouped['RETIRO_VOLUNTARIO'] = grouped['RETIRO_VOLUNTARIO'].map({False: 'No', True: 'Sí'})

# Paso 5: Asegurar que el código de área sea string
grouped['CODIGO_AREA'] = grouped['CODIGO_AREA'].astype(str)

# Paso 6: Crear el gráfico
fig = px.bar(
    grouped,
    x='CODIGO_AREA',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Códigos de Área vs Retiro Voluntario',
    labels={
        'CODIGO_AREA': 'Código de Área',
        'Cantidad': 'Número de Retiros',
        'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'
    },
    barmode='stack'
)

# Paso 7: Ajustar el ángulo de las etiquetas
fig.update_layout(xaxis_tickangle=-45)

# Mostrar el gráfico
fig.show()


In [ ]:
import plotly.express as px

# Paso 1: Top 20 áreas por cantidad de retiros
top_areas = retiros_reducido['AREA'].value_counts().head(20).index

# Paso 2: Filtrar el DataFrame por las top 20 áreas
df_top = retiros_reducido[retiros_reducido['AREA'].isin(top_areas)]

# Paso 3: Agrupar por AREA y RETIRO_VOLUNTARIO
grouped = df_top.groupby(['AREA', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 4: Asegurar que AREA esté como string (por si acaso)
grouped['AREA'] = grouped['AREA'].astype(str)

# Paso 5: Crear el gráfico apilado
fig = px.bar(
    grouped,
    x='AREA',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Áreas vs Retiro Voluntario',
    labels={
        'AREA': 'Área',
        'Cantidad': 'Número de Retiros',
        'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'
    },
    barmode='stack'  # Apilado
)

# Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1500,  # Ancho del gráfico
    height=800   # Alto del gráfico
)

fig.show()


In [ ]:
import plotly.express as px

# Paso 1: Top 20 códigos de cargo
top_cargos = retiros_reducido['CODIGO_CARGO'].value_counts().head(20).index

# Paso 2: Filtrar el DataFrame por los top 20 códigos de cargo
df_top = retiros_reducido[retiros_reducido['CODIGO_CARGO'].isin(top_cargos)]

# Paso 3: Agrupar por código de cargo y retiro voluntario
grouped = df_top.groupby(['CODIGO_CARGO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 4: Asegurar que CODIGO_CARGO esté como string para que sea categórico en el eje X
grouped['CODIGO_CARGO'] = grouped['CODIGO_CARGO'].astype(str)

# Paso 5: Crear el gráfico apilado
fig = px.bar(
    grouped,
    x='CODIGO_CARGO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Códigos de Cargo vs Retiro Voluntario',
    labels={'CODIGO_CARGO': 'Código de Cargo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack'  # Apilado
)

# Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1500,  # Ancho del gráfico
    height=800   # Alto del gráfico
)

fig.show()

In [ ]:
# Mostrar la tabla con el top 20 de códigos de cargo y retiro voluntario
tabla_top20 = grouped.sort_values(by='Cantidad', ascending=False)
display(tabla_top20.sort_index())


In [ ]:
import plotly.express as px

# Paso 1: Top 20 cargos por cantidad de retiros
top_cargos = retiros_reducido['CARGO'].value_counts().head(20).index

# Paso 2: Filtrar el DataFrame por los top 20 cargos
df_top = retiros_reducido[retiros_reducido['CARGO'].isin(top_cargos)]

# Paso 3: Agrupar por CARGO y RETIRO_VOLUNTARIO
grouped = df_top.groupby(['CARGO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 4: Asegurar que CARGO esté como string para que sea categórico en el eje X
grouped['CARGO'] = grouped['CARGO'].astype(str)

# Paso 5: Crear el gráfico apilado
fig = px.bar(
    grouped,
    x='CARGO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Top 20 Cargos vs Retiro Voluntario',
    labels={'CARGO': 'Cargo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack'  # Apilado
)

# Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1500,  # Ancho
    height=800   # Alto
)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Paso 1: Reemplazar los nombres de los meses con su formato en inglés
month_translation = {
    'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',
    'jul': 'Jul', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
}

# Asegurarse de que los valores de la columna 'PERIODO' estén en minúsculas
retiros_reducido['PERIODO'] = retiros_reducido['PERIODO'].str.lower()

# Reemplazar los meses con su formato en inglés
retiros_reducido['PERIODO'] = retiros_reducido['PERIODO'].replace(month_translation, regex=True)

# Paso 2: Convertir la columna PERIODO a formato datetime (ahora con meses en inglés)
retiros_reducido['PERIODO'] = pd.to_datetime(retiros_reducido['PERIODO'], format='%b-%y', errors='coerce')

# Paso 3: Verificar si hay valores nulos después de la conversión
print(retiros_reducido[retiros_reducido['PERIODO'].isna()])

# Paso 4: Calcular el número de bins según la regla de Freedman-Diaconis
Q1 = retiros_reducido['PERIODO'].quantile(0.25)
Q3 = retiros_reducido['PERIODO'].quantile(0.75)
IQR = Q3 - Q1
bin_width = 2 * IQR / (len(retiros_reducido['PERIODO']) ** (1 / 3))
bins = int((retiros_reducido['PERIODO'].max() - retiros_reducido['PERIODO'].min()) / bin_width)

# Paso 5: Crear el gráfico apilado con la variable RETIRO_VOLUNTARIO
fig = px.histogram(
    retiros_reducido,
    x='PERIODO',
    color='RETIRO_VOLUNTARIO',  # Apilar según RETIRO_VOLUNTARIO
    nbins=bins,  # Número de bins calculado
    title='Histograma de Frecuencia de Periodo por Retiro Voluntario',
    labels={'PERIODO': 'Periodo', 'count': 'Frecuencia'},
    barmode='stack',  # Apilado
    color_discrete_map={  # Asignar colores específicos
        False: 'blue',    # Falso será azul
        True: 'orange'    # Verdadero será naranja
    }
)

# Paso 6: Agregar contornos a las barras y modificar el tamaño del gráfico
fig.update_traces(marker_line_width=2, marker_line_color='black')  # Contornos de las barras en negro

# Paso 7: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,
    width=1000,  # Cambiar el ancho según desees
    height=600   # Cambiar la altura según desees
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Agrupar por la variable COMPAÑÍA y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['COMPAÑÍA', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 2: Asegurarse de que la columna COMPAÑÍA esté como string (si es necesario)
grouped['COMPAÑÍA'] = grouped['COMPAÑÍA'].astype(str)

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='COMPAÑÍA',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Compañía y Retiro Voluntario',
    labels={'COMPAÑÍA': 'Compañía', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1600,  # Ancho del gráfico
    height=900   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Agrupar por la variable REGION y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['REGION', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 2: Asegurarse de que la columna REGION esté como string (si es necesario)
grouped['REGION'] = grouped['REGION'].astype(str)

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='REGION',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Región y Retiro Voluntario',
    labels={'REGION': 'Región', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()


Pasos para un Feature Engineering

In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Agrupar por la variable FORMA_DE_TRABAJO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['FORMA_DE_TRABAJO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 2: Asegurarse de que la columna FORMA_DE_TRABAJO esté como string (si es necesario)
grouped['FORMA_DE_TRABAJO'] = grouped['FORMA_DE_TRABAJO'].astype(str)

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='FORMA_DE_TRABAJO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Forma de Trabajo y Retiro Voluntario',
    labels={'FORMA_DE_TRABAJO': 'Forma de Trabajo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna AÑO sea de tipo string (si es necesario)
retiros_reducido['AÑO'] = retiros_reducido['AÑO'].astype(str)

# Paso 2: Agrupar por la variable AÑO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['AÑO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='AÑO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Año y Retiro Voluntario',
    labels={'AÑO': 'Año', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna GÉNERO sea de tipo string (si es necesario)
retiros_reducido['GÉNERO'] = retiros_reducido['GÉNERO'].astype(str)

# Paso 2: Agrupar por la variable GÉNERO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['GÉNERO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='GÉNERO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Género y Retiro Voluntario',
    labels={'GÉNERO': 'Género', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna NIVEL_DEL_CARGO sea de tipo string (si es necesario)
retiros_reducido['NIVEL_DEL_CARGO'] = retiros_reducido['NIVEL_DEL_CARGO'].astype(str)

# Paso 2: Agrupar por la variable NIVEL_DEL_CARGO y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['NIVEL_DEL_CARGO', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='NIVEL_DEL_CARGO',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Nivel del Cargo y Retiro Voluntario',
    labels={'NIVEL_DEL_CARGO': 'Nivel del Cargo', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Paso 1: Asegurarse de que la columna RED_DE_SUCURSALES sea de tipo string (si es necesario)
retiros_reducido['RED_DE_SUCURSALES'] = retiros_reducido['RED_DE_SUCURSALES'].astype(str)

# Paso 2: Agrupar por la variable RED_DE_SUCURSALES y RETIRO_VOLUNTARIO
grouped = retiros_reducido.groupby(['RED_DE_SUCURSALES', 'RETIRO_VOLUNTARIO']).size().reset_index(name='Cantidad')

# Paso 3: Crear el gráfico de barras apiladas
fig = px.bar(
    grouped,
    x='RED_DE_SUCURSALES',
    y='Cantidad',
    color='RETIRO_VOLUNTARIO',
    title='Cantidad de Retiros por Red de Sucursales y Retiro Voluntario',
    labels={'RED_DE_SUCURSALES': 'Red de Sucursales', 'Cantidad': 'Número de Retiros', 'RETIRO_VOLUNTARIO': '¿Retiro Voluntario?'},
    barmode='stack',  # Apilado
)

# Paso 4: Modificar el tamaño del gráfico
fig.update_layout(
    xaxis_tickangle=-45,  # Rotar las etiquetas del eje X
    width=1200,  # Ancho del gráfico
    height=700   # Alto del gráfico
)

# Mostrar el gráfico
fig.show()

In [ ]:
# base libraries for data science
from pathlib import Path

import pandas as pd
import sklearn as sk
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [ ]:
retiros.info()

ELIMINACIÓN DE COLUMNAS NO NECESARIAS PARA EL MODELO

In [ ]:
variables_modelo = [
    "COMPAÑÍA", "CODIGO_AREA", "AREA", "CODIGO_CARGO", "CARGO", "REGION",
    "FORMA_DE_TRABAJO", "AÑO", "PERIODO", "GÉNERO", "RETIRO_VOLUNTARIO", "NIVEL_DEL_CARGO","RED_DE_SUCURSALES"
]

Estas variables describen al empleado antes del retiro y son apropiadas para entrenar el modelo:

COMPAÑÍA, CODIGO_AREA, AREA, CODIGO_CARGO, CARGO, REGION, FORMA_DE_TRABAJO, AÑO, PERIODO, GÉNERO.

El resto de variables se eliminan de la variable retiro_reducido, por qué no aporta información pertinente para el modelo o estas variables reflejan la decisión o consecuencia del retiro, o están muy relacionadas con la definición directa del target, por lo que confundiria el modelo.

In [ ]:
retiros_reducido = retiros[variables_modelo].copy()
retiros_reducido

In [ ]:
retiros_reducido.info()

VALORES FALTANTES

In [ ]:
retiros_reducido.isna().sum()

DATOS DUPLICADOS

In [ ]:
filas_duplicadas = retiros_reducido.duplicated().sum()
print("Cantidad de filas duplicadas: ", filas_duplicadas)

Se identifica que al renombrar el dataframe y manejar solo las variables necesarias para el modelo, tenemos información duplicada, pero esto se debe a que se elimina el ID; por lo que es normal que se repitan las filas con datos comunes entre sí.

In [ ]:
retiros_reducido.info()

INGENIERIA DE DATOS

In [ ]:
# Codificar la variable objetivo

retiros_reducido["RETIRO_VOLUNTARIO"] = retiros_reducido["RETIRO_VOLUNTARIO"].astype("int")               #.astype, se utiliza para cambiar el tipo de dato

retiros_reducido.sample(5)

In [ ]:
# Categoricas ordinales y target

target = 'RETIRO_VOLUNTARIO'

cols_categoric_ordinal = ["AÑO"]
print(cols_categoric_ordinal)

In [ ]:
# Columnas categóricas (no ordenadas)

cols_categoric_nominales = retiros_reducido.select_dtypes(include=['object', 'category']).columns.tolist()   #se puede hacer con el código de las ordinales o con este, llamando directamente los object y categoricos
cols_categoric_nominales

In [ ]:
# Columnas fecha
cols_fecha = retiros_reducido.select_dtypes(include=['datetime64[ns]']).columns.tolist()
cols_fecha

# **PIPELINES**

In [ ]:
categorical_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder()),
    ]
)

In [ ]:
categorical_ord_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder()),
    ]
)

# **PREPROCESADOR**

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[        ("categoric", categorical_pipe, cols_categoric_nominales),
        ("categoric ordinales", categorical_ord_pipe, cols_categoric_ordinal),
    ]
)

preprocessor

# **Train / Test split**

In [ ]:
# conjunto de entrenamiento y de prueba

X_features = retiros_reducido.drop("RETIRO_VOLUNTARIO", axis="columns")
Y_target = retiros_reducido["RETIRO_VOLUNTARIO"]

In [ ]:
# 80% train, 20% test

x_train, x_test, y_train, y_test = train_test_split(
    X_features, Y_target, test_size=0.2, stratify=Y_target
)

In [ ]:
# Verificamos dimensiones

x_train.shape, y_train.shape

In [ ]:
# Verificamos dimensiones

x_test.shape, y_test.shape

# **Preprocesamiento**

In [ ]:
transformed_data = preprocessor.fit(x_train)

transformed_data

In [ ]:
feature_names = preprocessor.get_feature_names_out()

feature_names

Para ver con más detalle las transformaciones del one hote encoding, podemos convertir rápidamente en un dataframe las variables preprocesadas: